<a href="https://colab.research.google.com/github/MartinSantaGitHub/python-ml-course/blob/master/code/T11%20-%201%20-%20TensorFlow101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a Tensor Flow

In [5]:
%tensorflow_version 1.x

In [6]:
import tensorflow as tf

In [7]:
print(tf.__version__)

1.15.2
